In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [4]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [6]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=True,
    batch_size=8
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [7]:
from modules.models.bert_models import BERTBiLSTMAttnCRF

In [9]:
model = BERTBiLSTMAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, is_freeze=False)

In [10]:
from modules.train.train import NerLearner

In [11]:
num_epochs = 100

In [12]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnCRF-fit_BERT-IO.cpt",
    t_total=num_epochs * len(data.train_dl))

In [13]:
model.get_n_trainable_params()

180482937

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [2]:
from modules.data import bert_data

In [5]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=False
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [6]:
from modules.models.bert_models import BERTBiLSTMAttnCRF

In [7]:
model = BERTBiLSTMAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, is_freeze=False)

In [8]:
from modules.train.train import NerLearner

In [9]:
num_epochs = 100

In [10]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnCRF-fit_BERT-IO.cpt")

In [11]:
model.get_n_trainable_params()

180482937

In [12]:
learner.load_model()

### Predict

In [13]:
from modules.data.bert_data import get_data_loader_for_predict

In [14]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [15]:
preds = learner.predict(dl)

In [16]:
from sklearn_crfsuite.metrics import flat_classification_report

In [17]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [18]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [19]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [20]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.7280    0.7977    0.7612      3865
       I_PER     0.9743    0.8447    0.9049      2112
       I_LOC     0.8222    0.8407    0.8314      1557

   micro avg     0.8065    0.8198    0.8131      7534
   macro avg     0.8415    0.8277    0.8325      7534
weighted avg     0.8165    0.8198    0.8160      7534



In [21]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [22]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.7537    0.7485    0.7511      1304
           O     0.9702    0.9809    0.9755     48370
         ORG     0.5860    0.4961    0.5374      1812
         PER     0.8207    0.6628    0.7333      1278

    accuracy                         0.9508     52764
   macro avg     0.7827    0.7221    0.7493     52764
weighted avg     0.9480    0.9508    0.9490     52764

